In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import datetime
import os
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as pe
from fbprophet import Prophet
for dirname, _, filenames in os.walk('/datathon'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\Wendy W\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: No module named 'fbprophet'

# Exloratory Data Analysis
### Initial Exploration
- Display the data and see if anything need to be done 
- Get the basic info about the data. How many rows and columns? What are the averages?
### Cleaning the data
- Change any datetime strings into datetime objects
- Transform rows and columns if must. 
- Interpret the meaning of each set of missing values and impute them.
### Plotting the data
- Use charts to see if there are any trends
- Compare and contrast between different groups of data: which have higher average?


In [ ]:
# Display the 3 dataframes
production = pd.read_csv("/kaggle/input/datathon/Total_Electricity_Production.csv")
print(production.shape)
display(production)

indicators = pd.read_csv("/kaggle/input/datathon/world_indicators.csv")
print(indicators.shape)

display(indicators)

history = pd.read_csv("/kaggle/input/datathon/south_africa_load_shedding_history.csv")
print(history.shape)

display(history)


# What need to be done & get noticed:
## Production
1. Datetime transformation
2. Column name renaming
3. Plot the production to see if there are any trends.
4. This time series is not continous. The measurements are taken 4 times a year. 

# Indicators
1. Some recent years are empty valued
2. Column names need to be renamed
3. Some columns and rows are not useful for us, and thus they need to be dropped.
4. We should implement a group by operation on this df on "country code" or country name
5. `country_code` and `country_name` are the same thing in this df, so one should be dropped. Considering we don't know codes that well, country_name is a better choice to be kept. 
6. Same applies to `series_name` and `series_code`.

# History
1. `created_at` is datetime string and should be converted to datetime objects.
2. The time series is not measured with the same time interval. There might be multiple blackouts per day, or there might be no blackouts at all, and those days are missing in the datetime column. 
3. Group by dates and either count the number of blackouts, or the average stage level on that day, or the maximum stage level on that day.
4. We need to impute these missing values with stage level 0 (that means no blackout)
5. P.S. Higher stage level indicates more electricity is being shedded. And the amount of electricity is proportional to the stage level number. So there is no need to transform this feature into the electricity corresponding to the stage level. Read more about the meaning of each load shedding level here: 



In [ ]:
production = pd.read_csv("/kaggle/input/datathon/Total_Electricity_Production.csv")

production.columns = ["datetime","production"]
production["datetime"] = pd.to_datetime(production["datetime"])
production["year"] = production["datetime"].apply(lambda x: x.year)
production["month"] = production["datetime"].apply(lambda x: x.month)
production["date"] = production["datetime"].apply(lambda x: x.date())
production["day_of_week"] = production["datetime"].apply(lambda x: x.day)
production["week"] = production["datetime"].apply(lambda x: x.week)
import plotly.graph_objects as go
fig=go.Figure()


fig.add_trace(go.Scatter(x=production["datetime"], y=production["production"],
                    mode='lines+markers',
                    name='Production'))
fig.update_layout(title="Total Electricity Production in South Africa, exisiting data",
                  xaxis_title="Datetime",yaxis_title="Giggawatts Hours")

fig.show()
elec = production.copy()
display(production)

In [ ]:
fig=go.Figure()


fig.add_trace(go.Scatter(x=production.loc[production["month"] == 7]["datetime"], y=production.loc[production["month"] == 7]["production"],
                    mode='lines+markers',
                    name='Production'))
fig.update_layout(title="Total Electricity Production in South Africa July of Each Year, exisiting data",
                  xaxis_title="Datetime",yaxis_title="Giggawatts Hours")

fig.show()
elec = production.copy()

# Interpretation of the Electricity Production Plot
- There is a yearly seasonality and a clear upward trend from 1985 to 2008-ish. There is a noticeable dip in 2009/01/01. And the electricity production is low-key decreasing after 2009. 
## Inspirtation questions:
1. What happend in 2009 January and a little before that?
2. What is the climate like in South Africa? Can the climate change explain the electricity production yearly seasonality?


# After Some research...
## 1. Analysis on the "Dip" in 2009
- Refer to the WikiPedia Site for more details : https://en.wikipedia.org/wiki/South_African_energy_crisis
- Basically, the first energy shortage emerges in late 2007 and lasted until at least May 2008. "The problem is related to the supply of coal to the coal-fired power plants.[11][12] Several other causes have been postulated, including skills shortages[4] and increasing demand for electricity around the country"
- Before the crisis, South Africa has been exporting its electricity to neighboring African states.
- On January 2008, South Africa delares to cease exporting power. 
- If we only look at electricity production in July of each year, then the pattern is clearer. Before Jul 2007, the trend is upward with 3-4 small drops. Since Jul 2007, the trend has been downard and with another "dip" in 2015. (2015 is the second period of load shedding after 2007-2008)

## 2. Analysis of the seasonality
- Since South Africa is in Southern Hemisphere so the seasonality should be the opposite of what the Northern Hemisphere has.
![image](/kaggle/input/datathon/south_africa_temperature.png)

- This image is the temperature model in South Africa over the years.
- The lowest temperature is in July (corresponding to highest electricity Production) averaged at 30 F, and the highest temperature is in January (corresponding to low electricity production) averaged around 90 F. Extreme weather at either end requires more electricity to generate heaters or coolers. There is an association between the production of electricity and the temperature.


# Interpolate Empty Values
- I will be using Facebook's Prophet to predict electricity production in 2009, 2020 and 2021.
- In order for the algorithm to work, the time series data has to be daily or monthly. 
- I am using interpolation to fill missing gaps between each measurement (Initially 4 times a year). 

In [ ]:
production_by_day = pd.DataFrame(index = pd.date_range(start='1985-01-01',end="2018-07-01",freq='D'))
production_by_day["production"] = production_by_day.index
elec
production_dict = production[["datetime","production"]].set_index("datetime")["production"].to_dict()
production_by_day["production"] = production_by_day["production"].replace(production_dict)
production_by_day["production"] = production_by_day["production"].apply(lambda x:np.nan if (type(x) != float) else x)
#df["production"] = df['production'].interpolate(method='time', inplace=True)
#df["production"] 
production_by_day["production"].interpolate(method='time', inplace=True)
production_by_day["production"]
import plotly.graph_objects as go
fig=go.Figure()


fig.add_trace(go.Scatter(x=production_by_day.index, y=production_by_day["production"],
                    mode='lines+markers',
                    name='Production'))
fig.update_layout(title="Total Electricity Production in South Africa,estimated using imputation",
                  xaxis_title="Datetime",yaxis_title="Giggawatts Hours")

fig.show()

# Electricity Production Estimate
- To check whether there is a link between electricity production and blackouts pattern, we need data
- Right now we only have production data up to 2018/7/01 and only 4 entries a year.
- There is definitely a seasonality to the data from the chart plotted above.
- First, I imputed the values using interpolation.
- Then I used FB's Prophet time series prediction algorithm to estimate production of between 2018/07/01 to 2020/09/07  as part of the training dataset, in order to build a prediction model for the blackouts.
- Making predictions after 2020/09/07 as features in test dataset. 

# Facebook Prophet
- Read more about this Algorithm here: https://facebook.github.io/prophet/docs/quick_start.html#python-api
- Basically, this is an open-source algorithm provided by Facebook that is ideal to model time series with seasonality and trends. 
- The training data have to have columns called `ds` and `y`. The `ds` column is datetime objects, and `y` column is the values we wish to predict.
- In our case, `y` is the electricity production in Gigawatts Hours.

In [ ]:
X = production_by_day.reset_index()
X.columns = ["ds","y"]
m = Prophet(weekly_seasonality=False)
m.add_seasonality(name='yearly', period=365, fourier_order=10)
test = pd.DataFrame({"ds":pd.date_range(start='2018-07-02',end="2021-09-07",freq='D')})

elec_forecast = m.fit(X).predict(test)
#fig = m.plot_components(forecast)


In [ ]:

#from fbprophet.plot import plot_plotly, plot_components_plotly
m.plot(elec_forecast)

# Prediction From Prophet
Above is the prediction given by Prophet. The prediction still has the yearly seasonality, but with a slight trend downward. 


# Moving on to the Next

In [ ]:
indicators = pd.read_csv("../input/datathon/world_indicators.csv")
indicators.columns = indicators.columns.str.lower()
indicators = indicators.applymap(lambda s:s.lower() if type(s) == str else s)
display(indicators.loc[225:232])
indicators = indicators.drop(range(228,233),axis=0)
indicators = indicators.drop(["series code","country code"],axis=1)
indicators.columns = indicators.columns.str.replace("\s\[.*\]","")

tuples = zip(indicators["country name"],indicators["series name"])
indicators.index = pd.MultiIndex.from_tuples(tuples, names=['country', 'indicator'])
indicators=indicators.drop(["country name","series name"],axis=1)
indicators

In [ ]:
# A function that takes information of a particular country of particular series
def get_country_df(country_name,df,col):
    
    
    sa_indicators = df.loc[country_name]
    sa_indicators=sa_indicators.transpose()
    return sa_indicators[col].dropna()
ser = get_country_df("south africa",indicators,"access to electricity (% of population)")
indicators.index.get_level_values(1).unique()

In [ ]:
country_names = np.unique(indicators.index.get_level_values(0))
fig = go.Figure()
graph_names=["access to electricity, rural (% of rural population)",
            "access to electricity, urban (% of urban population)",
            "access to electricity (% of population)"]
for name in graph_names:
    ser = get_country_df("south africa",indicators,name)
    fig.add_trace(go.Scatter(y=ser.values, x=ser.index,
                    mode='lines+markers',
                    name=name))
fig.update_layout(title = "access to electricity of South Africa",xaxis_title = "year",yaxis_title="percentage")

fig.show()
# comparison of Overall Access

fig = go.Figure()

for country in country_names:
    ser_access = get_country_df(country,indicators,"access to electricity (% of population)")
    #ser_rural_access = get_country_df(country,indicators,"access to electricity, rural (% of rural population)")
    #ser_urban_access = get_country_df(country,indicators,"access to electricity, urban (% of urban population)")
    #ser_gdp = get_country_df(country,indicators,"gdp per capita (constant 2010 us$)")
    fig.add_trace(go.Scatter(y=ser_access.values, x=ser_access.index,
                    mode='lines+markers',
                    name=country))
fig.update_layout(title = "Access to Electricity (% of Population)",xaxis_title = "Year",yaxis_title="Percentage (%)")
fig.show()

# comparison of Urban Access

fig = go.Figure()

for country in country_names:
    ser = get_country_df(country,indicators,"access to electricity, urban (% of urban population)")
    fig.add_trace(go.Scatter(y=ser.values, x=ser.index,
                    mode='lines+markers',name=country))
fig.update_layout(title = "Access to Electricity, Urban (% of Urban Population)",xaxis_title = "Year",yaxis_title="Percentage (%)")
fig.show()

# comparison of Urban Access

fig = go.Figure()

for country in country_names:
    ser = get_country_df(country,indicators,"access to electricity, rural (% of rural population)")
    fig.add_trace(go.Scatter(y=ser.values, x=ser.index,
                    mode='lines+markers',name=country))
fig.update_layout(title = "Access to Electricity, Rural (% of Rural Population)",xaxis_title = "Year",yaxis_title="Percentage (%)")
fig.show()



# Urban vs Rural Access to Electricity
## The energy crisis affects different populations differently
- The shedding didn't affect the urban area in terms of electricity access, while electricity access in rural areas decreased in 2013,2015,2016,2017. 


In [ ]:
fig = go.Figure()

for country in country_names:
    ser = get_country_df(country,indicators,"time required to get electricity (days)")
    fig.add_trace(go.Scatter(y=ser.values, x=ser.index,
                    mode='lines+markers',
                    name=country))
fig.update_layout(title = "time required to get electricity (days) in South Africa and its neighbors",xaxis_title = "year",yaxis_title="days")

fig.show()



# time required to get electricity (days)	
- What does it mean-'Time required to get electricity is the number of days to obtain a permanent electricity connection. The measure captures the median duration that the electricity utility and experts indicate is necessary in practice, rather than required by law, to complete a procedure.'
- This data is only available from 2009 to 2019, while South Africa already encountered the energy crisis. So we don't know how it does before the crisis.

- In 2009-2014 South Africa had the highest number of days to get electricity. 
- In 2015, the number plummeted. What happened during that time is unknown. However, still, the time required to get electricity for South Africa is above the world's average. 

In [ ]:
fig = go.Figure()

for country in country_names:
    ser = get_country_df(country,indicators,"electricity production from oil, gas and coal sources (% of total)")
    fig.add_trace(go.Scatter(y=ser.values, x=ser.index,
                    mode='lines+markers',
                    name=country))
fig.update_layout(title = "Electricity Production From oil, Gas and Coal Sources (% of total)",xaxis_title = "year",yaxis_title="Percentage (%)")

fig.show()
fig = go.Figure()

for country in country_names:
    ser = get_country_df(country,indicators,"electricity production from renewable sources, excluding hydroelectric (% of total)")
    fig.add_trace(go.Scatter(y=ser.values, x=ser.index,
                    mode='lines+markers',
                    name=country))
fig.update_layout(title = "Electricity Production from Renewable Sources, Excluding Hydroelectric (% of total)",xaxis_title = "year",yaxis_title="Percentage (%)")

fig.show()


In [ ]:
ser1= get_country_df("world",indicators,"electricity production from renewable sources, excluding hydroelectric (% of total)").loc["2015"]
ser2= get_country_df("world",indicators,"electricity production from oil, gas and coal sources (% of total)").loc['2015']
# This dataframe has 244 lines, but 4 distinct values for `day`
pie = pd.DataFrame([float(ser1),float(ser2)])
pie.index = ["electricity production from renewable sources, excluding hydroelectric (% of total)","electricity production from oil, gas and coal sources (% of total)"]
pie.columns= ["percentage"]
fig = pe.pie(pie.reset_index(), values='percentage', names='index')
fig.update_layout(title="Eletricity Production Composition in 2015, World Average")
fig.show()

ser1= get_country_df("south africa",indicators,"electricity production from renewable sources, excluding hydroelectric (% of total)").loc["2015"]
ser2= get_country_df("south africa",indicators,"electricity production from oil, gas and coal sources (% of total)").loc['2015']
# This dataframe has 244 lines, but 4 distinct values for `day`
pie = pd.DataFrame([float(ser1),float(ser2)])
pie.index = ["electricity production from renewable sources, excluding hydroelectric (% of total)","electricity production from oil, gas and coal sources (% of total)"]
pie.columns= ["percentage"]
fig = pe.pie(pie.reset_index(), values='percentage', names='index')
fig.update_layout(title="Eletricity Production Composition in 2015, South Africa")
fig.show()


# From traditional energy to renewable energy
- Lack of alternative choices might be correlated with the power outages. 
- The inital power outages are due to problems with supply of coal. 
- South Africa primarily relies heavily on traditional energy sources. So any shortage of coal can lead to energy crisis. 
- We saw an trend of increase in 2014-2015, though still below the world's average, is sign of switching to renewable energy. 
- In 2014, the country received USD 5.5 billion towards renewable energy projects. Renewable energy in South Africa has the potential to increase access to electricity in rural areas because of its suitability for off-grid and small-scale solutions. 
- There are many obstacles that South Africa has to face on its journey to switch to more renewable energy. Read more about it here: https://en.wikipedia.org/wiki/Renewable_energy_in_South_Africa

# History of Load Shedding
- Eskom SePlush has provided a history of load-sheding in South Africa from 2015 to 2020. There were no load-shedding in 2016 and 2017. 

## Data cleaning & Processing
- As I said earlier, the measurements are taken with different intervals in between. So to standardlize the time series, I use daily/weekly and aggregate the "stages". 
- However, the stage itself is difficult to aggregate. I have two approaches: 
    1. Times the stage with 1000, that would be the energy in Megawatts corresponding to the stage. Then aggregate on sum of the energy, which would be the total amount of energy shedded on that day/week.
    2. Aggregate on the count of entries (if doesn't equal stage 0). That would constitute as the number of blackouts on that day/week.

- I chose the first one, because it accounts for the intensity of each blackout.


In [ ]:
history = pd.read_csv("../input/datathon/south_africa_load_shedding_history.csv")
history.columns = ["datetime","stage"]
history["datetime"] = pd.to_datetime(history["datetime"])
history["year"] = history["datetime"].apply(lambda x: x.year)
history["month"] = history["datetime"].apply(lambda x: x.month)
history["date"] = history["datetime"].apply(lambda x: x.date()).astype(str)
history["day_of_week"] = history["datetime"].apply(lambda x: x.day)
history["week"] = history["datetime"].apply(lambda x: x.week)

history_by_day = pd.DataFrame(index = pd.date_range(start='2015-01-09',end="2020-09-07",freq='D'))
history_dic = history.groupby("date").agg({"stage":"sum"})["stage"].to_dict()
history_by_day["1000_MW"] = history_by_day.index.astype(str)
history_by_day["1000_MW"] = history_by_day["1000_MW"].replace(history_dic)
history_by_day["1000_MW"] = history_by_day["1000_MW"].apply(lambda x:0 if (type(x) != int) else x)
display(history_by_day)
fig=go.Figure()
fig.update_layout(title="Energy to Be Shed Each Day from 2015 to 2020",
                 xaxis_title="Date",yaxis_title="Energy (MW)",legend=dict(x=0,y=1,traceorder="normal"))
fig.add_trace(go.Scatter(x=history_by_day.index,
                         y=history_by_day["1000_MW"] * 1000,
                    mode='lines+markers',
                    name='Outage'))


fig.show()


<h1>Energy Crisis In South Africa: what happened at each cohorts of blackouts?
    </h1>
    <a src =”https://en.wikipedia.org/wiki/South_African_energy_crisis”>Link to the website</a>
<p>Loading shedding startedThe Majuba power plant lost its capacity to generate power after a collapse of one of its coal storage silos on 1 November 2014. The Majuba power plant delivered approximately 10% of the country's entire capacity and the collapse halted the delivery of coal to the plant.[20] A second silo developed a major crack on 20 November causing the shut down of the plant again, this after temporary measures were instituted to deliver coal to the plant.[21]
    </p>

On 5 December 2014, Eskom started major stage three load shedding in South Africa after the shut down of two power plants on 4 November (of said year) due to diesel shortages. It was also reported that the Palmiet and Drakensberg Pumped Storage Schemes were also experiencing difficulties due to a depletion of water reserve to the Hydro plants. Stage three was the highest degree of load shedding then.[22]

On Thursday 4 November, Eskom fell 4,000 megawatts (5,400,000 hp) short of the country's electricity demand of 28,000 megawatts (38,000,000 hp). The power utility has the ability to produce 45,583 megawatts (61,128,000 hp) but could only supply 24,000 megawatts (32,000,000 hp) due to "planned and unplanned" maintenance. One turbine at Eskom’s Duvha Power Station is still out of commission due to an "unexplained incident" in March 2014.[23] Load shedding was scheduled to resume in February 2015, due to industry start up, after the December holiday period. on[](http://) 5 December 2014,

In [ ]:
# To see a smoother curve, we use 7-day rolling or 30-day rolling. 
fig=go.Figure()

fig.add_trace(go.Scatter(x=history_by_day.index, y=(history_by_day["1000_MW"].rolling(window=7).mean())*1000,
                    mode='lines+markers',
                    name='Weekly'))
fig.update_layout(title="7-Day Rolling Mean",
                 xaxis_title="Date",yaxis_title="Energy (MW)",legend=dict(x=0,y=1,traceorder="normal"))

fig.show()
fig=go.Figure()

fig.add_trace(go.Scatter(x=history_by_day.index, y=1000*history_by_day["1000_MW"].rolling(window=30).mean(),
                    mode='lines+markers',
                    name='Monthly'))

fig.update_layout(title="30-day Rolling Mean",
                 xaxis_title="Date",yaxis_title="Energy (MW)",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
history_count = pd.DataFrame(index = pd.date_range(start='2015-01-01',end="2020-09-07",freq='D'))
history_count["num_blackouts"] = history_count.index.astype(str)
dic_count= history.groupby("date").agg({"stage":"count"})["stage"].to_dict()
history_count["num_blackouts"] = history_count["num_blackouts"].replace(dic_count)

history_count["num_blackouts"] = history_count["num_blackouts"].apply(lambda x:0 if (type(x) != int) else x)
year_sum = history.groupby("year").agg({"stage":"count"})
year_sum.columns = ["num_blackouts_sum"]


year_sum = year_sum.reset_index()
#outage.groupby("year").agg({"stage":"count"})
year_sum.columns = ["year","Number of national blackouts"]
ax = sns.barplot(x="year", y="Number of national blackouts", data=year_sum)




In [ ]:
year_sum = history.groupby("year").agg({"stage":"sum"})
year_sum.columns = ["energy_load_in_MW"]
year_sum["energy_load_in_MW"] = year_sum["energy_load_in_MW"] * 1000

year_sum = year_sum.reset_index()
year_sum.columns = ["year","energy_load_in_MW"]
ax = sns.barplot(x="year", y="energy_load_in_MW", data=year_sum)

# Number of Blackouts VS Amount of Energy Load being shed
- In 2015, the total energy being shed and the number of shedding is the highest among the four years.
- From 2018-2020, the number of shedding increased as year progressed.
- In 2020, I noticed that the amount of energy load being shed has not increased. Yet that is partially because we are still in 2020, and the load shedding is on going. So it is highly likely by the end of the year, the load shedding energy amoung will surpass that of 2019. 

- If that is the case, that means the number of blackouts and the amount of energy load being shed are both increasing since 2018. 

In [ ]:
# Weekly Sum
history_week = history_by_day.reset_index().resample('W-Wed', label='right', closed = 'right', on='index').sum().reset_index()
fig=go.Figure()

fig.add_trace(go.Scatter(x=history_week["index"], y=history_week["1000_MW"] * 1000,
                    mode='lines+markers',
                    name='Weekly'))

fig.update_layout(title="National Energy Load to be Shed By Week",
                 xaxis_title="Date",yaxis_title="Energy (MW)",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

# Energy Shortage Prediction for the Following Year
- I will be using the estimated/predicted electricity production for the next year, from the Prophet Model I have just built earlier as features. The features include the trend and the y-hat (the actual prediction).

- The first prediction model is Gaussian HMM. 

- The second method is using peak estimation. Power shortages often come in "clusters". By calculating the number of "peaks", the average height of each "peak" and average duration of each "peak", then multiplying the three things together, we can get an estimate on the average number of blackouts to expect for every year. 


# Method 1: GaussianHMM
- First, use the predicted/estimated values of electricity production as features for the training, and testing sets
- The training set will contain data from June 2018 to September 2020.
- The testing set will contain data from Jun 2018 to September 2021, with data in September 2020 to September 2021 being 0.


In [ ]:
elec_forecast

In [ ]:
elec_features = elec_forecast[["ds","trend","yhat"]]
elec_features.set_index("ds")
test = pd.DataFrame({"ds":pd.date_range(start='2018-06-01',end="2021-09-07",freq='D')})

elec_forecast_df = m.predict(test)
elec_testing_features = elec_forecast_df[["ds","trend","yhat"]]
elec_testing_features.set_index("ds")



elec_training_features = elec_features[["ds","trend","yhat"]]
display(elec_training_features)
#elec_training_features.set_index("ds")

In [ ]:
df_training = pd.DataFrame(index = pd.date_range(start='2018-06-01',end="2020-09-07",freq='D'))

df_training["1000_MW"] = df_training.index.astype(str)
df_training["date"] = df_training.index
dic = history.groupby("date").agg({"stage":"sum"})["stage"].to_dict()
df_training["1000_MW"] = df_training["1000_MW"].replace(dic)
df_training["1000_MW"] = df_training["1000_MW"].apply(lambda x:0 if (type(x) != int) else x)
elec_training_features
training = pd.concat([df_training.reset_index(),elec_training_features],axis=1).drop(["index","ds"],axis=1)
#training = training.resample('W-Wed', label='right', closed = 'right', on="date").sum().reset_index()
training = training.drop("date",axis = 1).dropna()
training
#training.to_csv("training.csv",index=False)

In [ ]:

testing = elec_testing_features.copy()
testing["ds"] = 0
testing.columns = ["1000_MW","trend","yhat"]

combined = pd.concat([training,testing[830:1195]],ignore_index=True)
display(combined)

In [ ]:
from hmmlearn import hmm
import matplotlib.pyplot as plt
import warnings
regr = hmm.GaussianHMM(n_components = 7,n_iter = 1000)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    regr.fit(training)
#plt.plot(regr.predict(training))
plt.plot(training["1000_MW"])
#combined = combined.drop("ds",axis = 1)
plt.plot(regr.predict(combined))

predictions = regr.predict(combined)


In [ ]:
future_predictions = predictions[830:]
future_dates = elec_testing_features.loc[830:][["ds"]]
future_dates["preds"] = future_predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=future_dates.ds, y=future_dates.preds * 1000,
                    mode='lines+markers',
                    name='Production'))
fig.update_layout(title="2020-2021 Predicted Energy Being Shedded using HMM",
                  xaxis_title="Datetime",yaxis_title="Energy (MW)")

fig.show()

In [ ]:
# Sum of energy for next year, predicted
print("Predicted Amount of Energy Load to be shed for 2020-2021:")
print(str(np.sum(predictions[830:]) * 1000)+" MW")


# Second Method:
Estimation of yearly number of blackouts from previous data.

In [ ]:
past_3_years = history.loc[history["year"]>2015]
df_peak= pd.DataFrame(index = pd.date_range(start='2017-09-07',end="2020-09-07",freq='D'))

df_peak["num_blackouts"] = df_peak.index.astype(str)
dic = history.groupby("date").agg({"stage":"count"})["stage"].to_dict()
df_peak["num_blackouts"] = df_peak["num_blackouts"].replace(dic)
df_peak["num_blackouts"] = df_peak["num_blackouts"].apply(lambda x:0 if (type(x) != int) else x)
df_peak

In [ ]:
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks,peak_widths
df_peak = df_peak.reset_index()
x = df_peak["num_blackouts"]
peaks, _ = find_peaks(x, height=0)
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.plot(np.zeros_like(x), "--", color="gray")
plt.show()

In [ ]:
from scipy.signal import chirp, find_peaks, peak_widths
import matplotlib.pyplot as plt
x = df_peak["num_blackouts"]

peaks, _ = find_peaks(x)
results_half = peak_widths(x, peaks, rel_height=0.5)

results_full = peak_widths(x, peaks, rel_height=1)

plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.hlines(*results_half[1:], color="C2")
plt.hlines(*results_full[1:], color="C3")
plt.show()

In [ ]:
avg_blackouts_per_time = np.mean(df_peak.loc[df_peak["num_blackouts"]!=0]["num_blackouts"])
avg_blackout_length = np.mean(results_full[0])
past_peak_num = len(peaks)
avg_peak = np.mean(x[peaks].values)
#print(df_peak["num_blackouts"].sum())
print("Prediction for the total number of blackouts for next year:"+ str(past_peak_num*avg_blackout_length*avg_blackouts_per_time/3))